# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [1]:
!pip install -qU openai

In [2]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

## Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [3]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [4]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "SanthoshLLM" # @param {type: "string"}
instructions = "You are an expert in worldly things. You are smart. Be respectable and civil in your responses." # @param {type: "string"}
model = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [5]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [6]:
assistant

Assistant(id='asst_9UxVnWUqnmrAluiDN568fX7w', created_at=1713124502, description=None, file_ids=[], instructions='You are an expert in worldly things. You are smart. Be respectable and civil in your responses.', metadata={}, model='gpt-3.5-turbo', name='SanthoshLLM', object='assistant', tools=[])

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [7]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [8]:
thread

Thread(id='thread_os9lgm55dESgw8dMaRFKabEC', created_at=1713124578, metadata={}, object='thread')

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"How old are you?"
)

Again, let's examine our `message` object!

In [10]:
message

Message(id='msg_LN0yyiis692Njmw5HxcIbnkn', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How old are you?'), type='text')], created_at=1713124637, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_os9lgm55dESgw8dMaRFKabEC')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [11]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = "Be friendly in your conversations. Reply 'I have no Idea!!' if you are not confident" # @param {type: "string"}

Let's run our Thread!

In [12]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=instructions + " " + additional_instructions
)

Now that we've run our thread, let's look at the object!

In [13]:
run

Run(id='run_0RCdRtC8cCHv1i6MWJGeo52s', assistant_id='asst_9UxVnWUqnmrAluiDN568fX7w', cancelled_at=None, completed_at=None, created_at=1713124760, expires_at=1713125360, failed_at=None, file_ids=[], instructions="You are an expert in worldly things. You are smart. Be respectable and civil in your responses. Be friendly in your conversations. Reply 'I have no Idea!!' if you are not confident", last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_os9lgm55dESgw8dMaRFKabEC', tools=[], usage=None, temperature=1.0, max_completion_tokens=None, max_prompt_tokens=None, truncation_strategy={'type': 'auto', 'last_messages': None}, incomplete_details=None, response_format='auto', tool_choice='auto')

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [14]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [15]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [16]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [17]:
messages.data[0]

Message(id='msg_yj7UutcrSpfiUfA8LHszTgry', assistant_id='asst_9UxVnWUqnmrAluiDN568fX7w', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I have no age as I am a computer program created to assist you. How can I help you today?'), type='text')], created_at=1713124760, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_0RCdRtC8cCHv1i6MWJGeo52s', status=None, thread_id='thread_os9lgm55dESgw8dMaRFKabEC')

## Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Creating an Assistant with the Retriever Tool

The first thing we'll want to do is create an assistant with the Retriever tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data

First, we need some data. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [19]:
!wget https://www.gutenberg.org/files/84/84-h/84-h.htm -o frankenstein.html

Now we can upload our file!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [20]:
file_reference = client.files.create(
  file=open("frankenstein.html", "rb"),
  purpose='assistants'
)

Let's look at what our `file_reference` contains!

In [21]:
file_reference

FileObject(id='file-BVVGjxXvAO73DEdg7oq8rVJF', bytes=1167, created_at=1713125184, filename='frankenstein.html', object='file', purpose='assistants', status='processed', status_details=None)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Please pay attention to [pricing](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) and don't forget to delete your files when you're done!

In [22]:
assistant = client.beta.assistants.create(
  name=name + "+ Retrieval",
  instructions=instructions,
  model=model,
  tools=[{"type": "retrieval"}],
  file_ids=[file_reference.id]
)

Let's try submitting a message to our Assistant and seeing what kind of answer we get!

We'll outline the steps needed to do this in full:

1. Create an Assistant
2. Create a Thread
3. Add Messages to that Thread
4. Create a Run on that Thread
5. Wait for Run to Complete
6. Collect Messages from the Thread

Let's do that below!

In [23]:
# Create a Thread
thread = client.beta.threads.create()

# Add Messages to that Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What is the first words Victor Frankenstein speaks?"
)

# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress


Let's look at the final result!

In [24]:
messages

SyncCursorPage[Message](data=[Message(id='msg_Va1Zhp88TX2NQZ9hhdSaxxMd', assistant_id='asst_U1Y6HmeFkAvfNdvfoJnFImM9', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The first words that Victor Frankenstein speaks are contained in his letter to Mrs. Saville in England, which is the first entry in the novel. The specific words Victor speaks are not provided in the document. If you would like, I could continue searching for the exact text of the letter to Mrs. Saville, or if you have any other specific requests, feel free to let me know.'), type='text')], created_at=1713125273, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_43NjYn6XxA6zxSgTznVUf717', status=None, thread_id='thread_Wq0pBrj7KSpMQSyu3kCcostY'), Message(id='msg_P59To4brGTfyJc4UtdGqr2Ir', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the first words Victor Fra

Let's do some clean up to make sure we're not being charged anything extra by deleting our resources.

In [25]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

### Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [26]:
assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [27]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "file_ids": [file_reference.id]
    }
  ]
)

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [28]:
# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


We can check the specific steps that the Code Interpreter ran to figure out what steps the Assistant took!

In [29]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [30]:
for step in run_steps.data:
  print(step.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_8R90L7klzREpoKLQbtYKBmfW'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_kMXJHecMECikQWynPHO3yQMg', code_interpreter=CodeInterpreter(input="import mimetypes\r\n\r\nfile_path = '/mnt/data/file-BVVGjxXvAO73DEdg7oq8rVJF'\r\nmime_type, _ = mimetypes.guess_type(file_path)\r\n\r\nmime_type", outputs=[CodeInterpreterOutputLogs(logs='', type='logs')]), type='code_interpreter')], type='tool_calls')
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_dMAJCEd16tzZoj6v4RsGeunF'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_Nb0Z5A7rBqK4ugtnnKcdhpDp', code_interpreter=CodeInterpreter(input='import mimetypes\r\n\r\n# Get the mime type of the file\r\nmime_type, _ = mimetypes.guess_type(file_path)\r\n\r\nmime_type', outputs=[CodeInterpreterOutputLogs(logs="---------------------------------------------------------

In [31]:
messages

SyncCursorPage[Message](data=[Message(id='msg_8R90L7klzREpoKLQbtYKBmfW', assistant_id='asst_qmYxgsbxqJvCwvjK4JyGud6j', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="The uploaded file is identified as an 'image/vnd.dwg' file, which is a format typically used for Computer Aided Design (CAD) drawings. If you have any specific questions or tasks related to this file, feel free to let me know!"), type='text')], created_at=1713125409, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Zpiw2lJypi9BgLN7UfLMpt2B', status=None, thread_id='thread_tm4l1LoST8HbiOUB53vK8SEW'), Message(id='msg_dMAJCEd16tzZoj6v4RsGeunF', assistant_id='asst_qmYxgsbxqJvCwvjK4JyGud6j', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='It seems there was an error in retrieving the file type. Let me correct the code and try again.'), type='text')], created_at=1713125407, file_ids=[], incomp

In [32]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

NotFoundError: Error code: 404 - {'error': {'message': "No file found with id 'file-BVVGjxXvAO73DEdg7oq8rVJF'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [33]:
!pip install -qU duckduckgo_search

In [34]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [37]:
duckduckgo_search("What is the major city in Saturn?")

"Size and Distance. With an equatorial diameter of about 74,897 miles (120,500 kilometers), Saturn is 9 times wider than Earth. If Earth were the size of a nickel, Saturn would be about as big as a volleyball. From an average distance of 886 million miles (1.4 billion kilometers), Saturn is 9.5 astronomical units away from the Sun.\nSaturn is the sixth planet from the Sun and the second-largest in the Solar System, after Jupiter.It is a gas giant with an average radius of about nine-and-a-half times that of Earth. It has only one-eighth the average density of Earth, but is over 95 times more massive. Even though Saturn is nearly the size of Jupiter, Saturn has less than one-third of Jupiter's mass.\nBasic astronomical data. Saturn orbits the Sun at a mean distance of 1,427,000,000 km (887 million miles). Its closest distance to Earth is about 1.2 billion km (746 million miles), and its phase angle—the angle that it makes with the Sun and Earth—never exceeds about 6°. Saturn seen from t

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [38]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}

####❓ Question

Why does the description key-value pair matter?

#### Answer

The LLM takes the hint to use and tool or not based on the string given in the description.

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [39]:
assistant = client.beta.assistants.create(
    name=name + " + Function Calling API",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

We need to make a few modifications to our Assistant to include the ability to make calls to our local function and pass the results back to our Assistant for further generation.

In [40]:
import json

def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "duckduckgo_search":
            print("Consulting Duck Duck Go...")
            output = duckduckgo_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    print(tool_output_array)

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

def use_assistant(query, assistant_id, thread_id=None):
  thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=query,
  )

  print("Creating Assistant ")

  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  print("Querying OpenAI Assistant Thread.")

  run = wait_for_run_completion(thread.id, run.id)

  if run.status == 'requires_action':
    run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
    run = wait_for_run_completion(thread.id, run.id)

  print_messages_from_thread(thread.id)

  return thread.id

####❓ Question

Outline, in simple terms, what the `use_assistant` helper function is doing.

#### Answer
use_assistant function is creating a message, attaching it to the thread and running the LLM. If the LLM says needs help or says "requires_attention", it is checking with the duckduckgo search engine and returning result after it's done(completed/failed/requires attention). Finally printing the response and returning the thread_id

In [41]:
use_assistant("Who is the current Captain of the Mumbai Indians?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_RmfXydB57VQ1zyxsJ3fSIq5k', 'output': 'About two months ago, Mumbai Indians announced Hardik Pandya as their new captain for IPL 2024, marking the end of Rohit Sharma\'s legendary reign at the man in charge. As expected, the decision ...\nMumbai Indians Captain Hardik Pandya will return to Wankhede Stadium for the first time as Mumbai Indians captain on Monday. ... was sent near the boundary rope, to the surprise of the current India captain. Another time, Hardik kept gesturing to Rohit to move around, leading Suresh Raina, on commentary to say, "Rohit, now Hardik is the ...\nBy Mumbai Indians | 15 Dec, 2023. Mumbai Indians today announced a significant leadership transition for the upcoming 2024 season. Renowned all-rounder Hardik Pandya is set to take the helm as the captain of the Mumbai Indians, succeeding its longest-serving, one of the most s

'thread_Vf2OPwyS51uyq7BH8Lpligjl'

## Wrapping it All Together

Now we can create an Assistant with all of the available tools and see how it responds to various queries!

In [42]:
assistant = client.beta.assistants.create(
    name=name + " + All Tools",
    instructions=instructions,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function" : ddg_function}
    ],
    model=model,
    file_ids=[file_reference.id],
)

In [43]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The current Captain of the Winnipeg Jets is Blake Wheeler【9:0†source】.
user: Who is the current Captain of the Winnipeg Jets?


'thread_BExBzY04MqxwDOI8z1oXCbPe'

In [44]:
use_assistant("Who is the author of the supplied file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The author of the supplied file is Mary Shelley. The file is the text of the book "Frankenstein" by Mary Shelley【5†source】.
user: Who is the author of the supplied file?


'thread_7bJ8X8NB0ayzObeNrMXtO5mY'

In [45]:
use_assistant("How many bytes is the provided file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: requires_action
[]


BadRequestError: Error code: 400 - {'error': {'message': "Expected tool outputs for call_ids ['call_jgbBDVVHV2B73sV4rbgd2f9U'], got []", 'type': 'invalid_request_error', 'param': None, 'code': None}}

####❓ Question

Notice that our response can go through multiple paths, given that:

What is "deciding" to use the tool?

#### Answer :
We can control with the description in the tool, to force it to alwaz use the tool or only in certail situaations.

There are a bunch of factors that decide to use the tool or not such as Task requirments, tool availability and tool suitability. Also the evaluator function can determine if the task is accomplished or not.


### Adding JSON Mode for More Agentic Behaviour

Finally, we have the ability to select tools - all we need to do now is set up a process to allow us to create some kind of loop and make decisions about whether or not the response is complete or not.

We'll leverage the OpenAI completions end-endpoint with JSON mode to let us understand when we've adequately answered our user's question!

In [46]:
completed_template = \
"""
Does this response adequately answer the user's query?

Please return your response in JSON format - with key: "completed" and either True (if completed) or False (if not completed)

User Query:
{query}

Assistant Response:
{response}
"""

def is_complete(query, response):
  completed_response = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": completed_template.format(query=query, response=response),
          }
      ],
      model=model,
      response_format={"type" : "json_object"}
  )

  return completed_response

In [47]:
query = "How many bytes is the provided file?"

thread_id_for_response = use_assistant(query, assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The provided file is 1167 bytes in size.
user: How many bytes is the provided file?


Now we can observe JSON mode in action!

In [48]:
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
response = messages.data[0].content[0].text.value
completed_flag = json.loads(is_complete(query, response).choices[0].message.content)

In [49]:
completed_flag

{'completed': True}

## 🚧 BONUS CHALLENGE 🚧:

Use the components we've constructed so far to build a loop that lets us continue to query the Assistant if the response is not completed!

In [ ]:
### YOUR CODE HERE

# Make Sure You Delete Resources

Make sure you delete all the resources you created!

This function will help you do so!

In [50]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)